In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import os
import json
from glob import glob
import yaml
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [ ]:
%cd /content/drive/MyDrive/AI/yolov5
!pip install -r requirements.txt

In [ ]:
obj = {
    0: 'Bench',
    1: 'BenchBack',
    2: 'Bollard',
    3: 'BoundaryStone',
    4: 'BrailleBlock',
    5: 'Manhole',
    6: 'ProtectionFence',
    7: 'RoadSaftySign',
    8: 'StreetLampPole',
    9: 'StreetTreeCover',
    10: 'Trench'
    }

In [ ]:
# %cd /content/drive/MyDrive/AI/data
# !unzip -qq "/content/drive/MyDrive/AI/sample_zip/ProtectionFence.zip"
# %cd /content
# os.getcwd()

In [ ]:
ROOT_PATH = "/content/drive/MyDrive/AI/"

DATA_PATH = ROOT_PATH+'/data/'

### 수정할 부분 ###
## train or val
name='ProtectionFence'
train_val = 'valid'
###################

## 크롭 이미지 폴더명 + train or valid
IMG_PATH = f'{DATA_PATH}/{train_val}/images/'

In [ ]:
%cd /content/drive/MyDrive/AI/yolov5
########################################## --name 수정
!python detect.py --source '{IMG_PATH}' --weights /content/drive/MyDrive/AI/weights/upgrade_facility_best.pt --img 416 --conf 0.2 --save-txt --nosave --exist-ok --name ProtectionFence_valid

In [ ]:
### class 및 train_val 수정
IMG_PATH = f'{DATA_PATH}/{train_val}/images/'
LABEL_PATH_1 = ROOT_PATH + 'yolov5/runs/detect/ProtectionFence_valid/labels/'
LABEL_PATH_2 = DATA_PATH + 'valid/labels/'
################################################3


images = []
labels_1 = []
labels_2 = []


img_list = glob(IMG_PATH + '*.jpg')
label_list_1 = glob(LABEL_PATH_1 + '*.txt')
label_list_2 = glob(LABEL_PATH_2 + '*.txt')

print(len(img_list))
print(len(label_list_1))
print(len(label_list_2))

for img in img_list:

    file_img_name=os.path.basename(img)[:-4]

    path_1 = LABEL_PATH_1 + file_img_name + '.txt'
    path_2 = LABEL_PATH_2 + file_img_name + '.txt'


    if path_1 in label_list_1 and path_2 in label_list_2:
        images.append(img)
        labels_1.append(path_1)
        labels_2.append(path_2)
print('-'*30)
print(len(images))
print(len(labels_1))
print(len(labels_2))

In [ ]:
# 저장폴더명
savefolder = 'cropped/valid/'

for i in range(len(images)):
    image=cv2.imread(images[i])
    height,width,c=image.shape
    file_img_name=os.path.basename(images[i])[:-5]

    boxes_1=[]
    boxes_2=[]

    label_path_1 = labels_1[i]
    label_path_2 = labels_2[i]

    image1 = Image.open(images[i])

    
    with open(label_path_1, "r") as f:
        lines=f.readlines()
    f.close()
    
    for line in lines:
        boxes_1.append(list(map(float, line.split())))
        
    with open(label_path_2, "r") as f:
        lines=f.readlines()
    f.close()
    
    for line in lines:
        boxes_2.append(list(map(float, line.split())))
        
        
    
    for i in range(len(boxes_1)):
        if obj[boxes_1[i][0]] != name:
            continue
            
        cx_1=width*boxes_1[i][1]
        cy_1=height*boxes_1[i][2]
        w_1=width*boxes_1[i][3]
        h_1=height*boxes_1[i][4]
        
        sx_1=int(cx_1-w_1/2)
        sy_1=int(cy_1-h_1/2)
        ex_1=int(cx_1+w_1/2)
        ey_1=int(cy_1+h_1/2)


        # 이미지 자르기 crop함수 이용 ex. crop(left,up, rigth, down)
        croppedImage=image1.crop((sx_1,sy_1,ex_1,ey_1))
    
        # crop된 이미지 저장
        PATH = ROOT_PATH + savefolder +'images/' + file_img_name + str(i) + '.jpeg'
        croppedImage.save(PATH, 'JPEG')

        for j in range(len(boxes_2)):
            cx_2 = width*boxes_2[j][1] - sx_1
            cy_2 = height*boxes_2[j][2] - sy_1
            w_2 = width*boxes_2[j][3]
            h_2 = height*boxes_2[j][4]

            boxes_2[j][0] = 0
            boxes_2[j][1] = cx_2 / croppedImage.size[0]
            boxes_2[j][2] = cy_2 / croppedImage.size[1]
            boxes_2[j][3] = w_2 / croppedImage.size[0]
            boxes_2[j][4] = h_2 / croppedImage.size[1]


        # 라벨 저장
        arr = np.array(boxes_2)
        PATH = ROOT_PATH + savefolder +'labels/' + file_img_name + str(i) + '.txt'
        np.savetxt(PATH, arr, fmt='%1.5f', delimiter=" ", newline="\n")